#  IN3050/IN4050 Mandatory Assignment 1: Traveling Salesman Problem


## Rules
Before you begin the exercise, review the rules at this website:
https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html
(This is an individual assignment. You are not allowed to deliver together or copy/share source-code/answers
with others.)

## Delivering
**Deadline**: *Friday, February 21, 2020*

## What to deliver?
Deliver one single zipped folder (.zip, .tgz or .tar.gz) which includes:
* PDF report containing:
    * Your name and username (!)
    * Instructions on how to run your program.
    * Answers to all questions from assignment.
    * Brief explanation of what you’ve done.
    * *Your PDF may be generated by exporting your Jupyter Notebook to PDF, if you have answered all questions in your notebook*
* Source code
    * Source code may be delivered as jupyter notebooks or python files (.py)
* The european cities file so the program will run right away.
* Any files needed for the group teacher to easily run your program on IFI linux machines.

**Important**: if you weren’t able to finish the assignment, use the PDF report to elaborate on what you’ve tried
and what problems you encountered. Students who have made an effort and attempted all parts of the assignment
will get a second chance even if they fail initially. This exercise will be graded PASS/FAIL.

## Introduction
In this exercise, you will attempt to solve an instance of the traveling salesman problem (TSP) using different
methods. The goal is to become familiar with evolutionary algorithms and to appreciate their effectiveness on a
difficult search problem. You may use whichever programming language you like, but we strongly suggest that
you try to use Python, since you will be required to write the second assignment in Python. You must write
your program from scratch (but you may use non-EA-related libraries).


|  &nbsp;   | Barcelona | Belgrade |  Berlin | Brussels | Bucharest | Budapest |
|:---------:|:---------:|:--------:|:-------:|:--------:|:---------:|:--------:|
| Barcelona |     0     |  1528.13 | 1497.61 |  1062.89 |  1968.42  |  1498.79 |
|  Belgrade |  1528.13  |     0    |  999.25 |  1372.59 |   447.34  |  316.41  |
|   Berlin  |  1497.61  |  999.25  |    0    |  651.62  |  1293.40  |  1293.40 |
|  Brussels |  1062.89  |  1372.59 |  651.62 |     0    |  1769.69  |  1131.52 |
| Bucharest |  1968.42  |  447.34  | 1293.40 |  1769.69 |     0     |  639.77  |
|  Budapest |  1498.79  |  316.41  | 1293.40 |  1131.52 |   639.77  |     0    |


<center>Figure 1: First 6 cities from csv file.</center>


## Problem
The traveling salesman, wishing to disturb the residents of the major cities in some region of the world in
the shortest time possible, is faced with the problem of finding the shortest tour among the cities. A tour
is a path that starts in one city, visits all of the other cities, and then returns to the starting point. The
relevant pieces of information, then, are the cities and the distances between them. In this instance of the
TSP, a number of European cities are to be visited. Their relative distances are given in the data file, *european_cities.csv*, found in the zip file with the mandatory assignment.

(You will use permutations to represent tours in your programs. If you use Python, the **itertools** module provides
a permutations function that returns successive permutations, this is useful for exhaustive search)

## Exhaustive Search
First, try to solve the problem by inspecting every possible tour. Start by writing a program to find the shortest
tour among a subset of the cities (say, **6** of them). Measure the amount of time your program takes. Incrementally
add more cities and observe how the time increases.

In [2]:
# Implement the algorithm here
import numpy as np
import pandas as pd
from itertools import permutations 
import time
import sys
import math
import statistics

#henter datasett
data = pd.read_csv("european_cities.csv",sep=";",usecols=range(0,24))


def avstand_fra_til(data,vei):
    '''
    regner ut avstand i ruten (score)
    
    args:
        data: datasett
        vei: rute/vei
        
    returns:
        avstand:avstand
        
    '''
    avstand=0
    for i in range(len(vei)-1):
        avstand+=data[vei[i],vei[i+1]]
        
    avstand+=data[vei[-1],vei[0]]
    
    return int(avstand)

def rand_seq(size):
    '''
    lager random sekvens, permutasjon
    
    args:
        size: lengde på sekvens
        seq: sekvens, permutasjon
    returns:
        seq: sekvens
    
    '''
    seq=np.array(np.arange(24))
    np.random.shuffle(seq)
    seq=seq[0:size]
    
    return seq




def tsp(data,byer=[],size=5):
    '''
    Exhaustive search
    
    args:
        data: datasett
        byer: stoppe sted for ruten
        size: lengden på ruten
    returns:
        out_avstand: avstand (fitness)
        out_byer: byer i ruten
        
    
    '''
    tid=time.time()

    sub_data=data.to_numpy()
    
    byer=rand_seq(size) if len(byer)==0 else byer
        
    combi=permutations(byer)

    short_distance=[1000000000000000000,byer]
    
    for vei in combi:
        
        avstand=avstand_fra_til(sub_data,vei)
        
        short_distance=[avstand,vei] if short_distance[0]>avstand else short_distance
    
    out_avstand=short_distance[0]
    out_byer=[list(data.columns)[i] for i in short_distance[1]]
    tid=time.time()-tid
           
    return out_avstand,out_byer


    

In [2]:
#tidtakning av exhaustive search
%timeit -n1 -r100 tsp(data,byer=[0,1,2,3,4,5])
%timeit -n1 -r100 tsp(data,byer=[0,1,2,3,4,5,6])
%timeit -n1 -r10 tsp(data,byer=[0,1,2,3,4,5,6,7])
%timeit -n1 -r2 tsp(data,byer=[0,1,2,3,4,5,6,7,8])
%timeit -n1 -r2 tsp(data,byer=[0,1,2,3,4,5,6,7,8,9])

4.82 ms ± 440 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)
22.1 ms ± 9.63 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)
274 ms ± 74.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
3.12 s ± 14.3 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
33 s ± 352 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


What is the shortest tour (i.e., the actual sequence of cities, and its length) among the first 10 cities (that is,
the cities starting with B,C,D,H and I)? How long did your program take to find it? Calculate an approximation of how long it would take to perform exhaustive search on all 24 cities?

In [3]:
# tid det for exhaustive search med 24 ruter
ti_f=math.factorial(10)
tjuefire_f=math.factorial(24)

#ca.tid per loop er 14.1s/10!
tid_per_loop=(14.1*1000)/ti_f  # i ms
#ca. tid for 24! loop
tid_tjuefire_f_loop=tjuefire_f*tid_per_loop/1000 # i sekunder
#i tilegg har avstandsfunksjonen for 24 byer ca.2 som mange operasjonen

tid=tid_tjuefire_f_loop*2/60/60/24/365 # loop(24!)*avstandsfunksjon/minutter/timer/dager/år

# tida det tar blir da ca. 152892132690 år, lenger tid en universet har eksistert
tid

152892132690.83176

In [4]:
# kortese rute
f=tsp(data,byer=[0,1,2,3,4,5,6,7,8,9])
print(f'korteste avstand: {f[0]}, rute: {f[1]}')

korteste avstand: 7486, rute: ['Barcelona', 'Belgrade', 'Istanbul', 'Bucharest', 'Budapest', 'Berlin', 'Copenhagen', 'Hamburg', 'Brussels', 'Dublin']


## Hill Climbing
Then, write a simple hill climber to solve the TSP. How well does the hill climber perform, compared to the result from the exhaustive search for the first **10 cities**? Since you are dealing with a stochastic algorithm, you
should run the algorithm several times to measure its performance. Report the length of the tour of the best,
worst and mean of 20 runs (with random starting tours), as well as the standard deviation of the runs, both with the **10 first cities**, and with all **24 cities**.

In [3]:
# Implement the algorithm here


def bytt_index(vei):
    '''
    bytter posisjonen to elementer i arrayen
    Args:
        vei: rute
    returns:
        vei: rute
    '''
    vei=vei.copy()
    
    i,j=np.random.choice(list(range(0,len(vei))),2,replace=False)
            
    vei[i],vei[j]=vei[j], vei[i]
    return vei
    
        

def hill_climb(data,size=5,byer=[],margin=40,max_range=100):
    '''
    hill climbing algoritme
    args:
        data:datasett
        size: rute lengde
        byer: byer i ruta
        margin: stansmargine for stans etter n(margin) loop uten ending i kortest avstand
        max_range: max iterasjoner
    returns:
        best_vei: kortest rute
        best_avstand: avstand på best_vei
        
    '''

    sub_data=data.to_numpy()
    
    byer=rand_seq(size) if len(byer)==0 else byer
      
    best_vei=byer.copy()
    best_vei_avstand=avstand_fra_til(sub_data,best_vei)
    
    count=0
    
    
    for i in range(max_range):
        
        ny_vei=bytt_index(best_vei)
        ny_avstand=avstand_fra_til(data.to_numpy(),ny_vei)
        
        if best_vei_avstand>ny_avstand:
            best_vei=ny_vei
            best_vei_avstand=ny_avstand
            count=0
        else:
            count+=1
            
        
        if count>margin:
            #print(f'{i}, og counten er {count}')
            break
        
                
                
    return best_vei_avstand, best_vei



In [282]:
avstander_ti=[]
avstander_tjuefire=[]
rute_ti=np.array([0,1,2,3,4,5,6,7,8,9])
rute_tjuefire=np.random.choice(list(range(0,24)),24,replace=False)

for i in range(20):
    byer_ti=np.roll(rute_ti,np.random.randint(0,10))
    byer_tjuefire=np.roll(rute_tjuefire,np.random.randint(0,10))
    avstander_ti.append(hill_climb(data,byer=byer_ti,margin=200,max_range=1000)[0])
    avstander_tjuefire.append(hill_climb(data,byer=byer_tjuefire,margin=200,max_range=1000)[0])

gjennomsnitt_10=statistics.mean(avstander_ti)
standardavvik_10=statistics.stdev(avstander_ti)
gjennomsnitt_24=statistics.mean(avstander_tjuefire)
standardavvik_24=statistics.stdev(avstander_tjuefire)
print("ti ruter, exhaustive search\n------------------")
print(f'beste avstand: {tsp(data,byer=rute_ti)[0]}\n')
print('Ti ruter \n-------------------')
print(f'Gjennomsnitt: {int(gjennomsnitt_10)} \nStandardavik: {int(standardavvik_10)}\
    \nMinst: {min(avstander_ti)}\nMax: {max(avstander_ti)}')
print('\nTjuefire ruter \n-------------------')
print(f'Gjennomsnitt: {int(gjennomsnitt_24)} \nStandardavik: {int(standardavvik_24)}\
    \nMinst: {min(avstander_tjuefire)}\nMax: {max(avstander_tjuefire)}')


    

ti ruter, exhaustive search
------------------
beste avstand: 7486

Ti ruter 
-------------------
Gjennomsnitt: 7613 
Standardavik: 220    
Minst: 7486
Max: 8419

Tjuefire ruter 
-------------------
Gjennomsnitt: 15137 
Standardavik: 886    
Minst: 13699
Max: 16853


In [299]:
%time hill_climb(data,byer=[0,1,2,3,4,5,6],margin=50,max_range=2000)

Wall time: 11 ms


(5487, [5, 4, 1, 0, 3, 6, 2])

## Genetic Algorithm
Next, write a genetic algorithm (GA) to solve the problem. Choose mutation and crossover operators that are appropriate for the problem (see chapter 4.5 of the Eiben and Smith textbook). Choose three different values for the population size. Define and tune other parameters yourself and make assumptions as necessary (and report them, of course).

For all three variants: As with the hill climber, report best, worst, mean and standard deviation of tour length out of 20 runs of the algorithm (of the best individual of last generation). Also, find and plot the average fitness of the best fit individual in each generation (average across runs), and include a figure with all three curves in the same plot in the report. Conclude which is best in terms of tour length and number of generations of evolution
time.

In [85]:

def muta(vei,p=0.2):
    '''
    swap_mutasjon
    args:
        vei: rute
        p: sannsylighet for mutasjon
    return:
        vei: rute
    '''
    endret=False
    if np.random.rand()<p:
        vei=bytt_index(vei)
        endret=True
    return vei,endret

def muta_list(veier,p=0.2):
    '''
    swap_mutasjon for a hole list
    args:
        vei: rute
        p: sannsynlighet for mutasjon
    return:
        ny_list: list with mutated paths
    '''
    endret_list=[]
    for i in veier:
        ny,endret=muta(i)
        if endret:
            endret_list.append(ny)
    ny_list=np.array(endret_list)
    
    return ny_list
    

def croosover(a,b):
    '''
    order crossover
    args:
        a: parent 1
        b: parent 2
    returns:
        kid: child
        
    '''
    a.tolist()
    b.tolist()
    lengde=len(a)
    start,slutt=np.random.choice(lengde,2,replace=False)
    
    if start>slutt:
        start,slutt=slutt,start 
        
    kid=[1000]*lengde
    kid[start:slutt]=a[start:slutt]
    ufylt_index=[i for i,j in enumerate(kid) if j==1000]
    b_igjen=[i for i in b if i not in kid]

    for index,val in zip(ufylt_index,b_igjen):
        kid[index]=val
        
    kid=np.array(kid,dtype=np.int)
        
    return kid

def fitness(data,veier):
    '''sjekker fitness, avstand for ruten
    arg:
        data: datasett
        veier: lister av ruter
    returns:
        veier: liste av ruter, sortert etter laveste avstand
        
    '''
    data=data.to_numpy()
    fit_poeng=np.array([avstand_fra_til(data,veier[i,:]) for i in range(veier.shape[0])])
    fit_sort=np.argsort(fit_poeng)
    veier=veier[:][fit_sort]
    
    return veier

def popin(byer,size):
    '''intialiserer en list med veier, uten noen duplikater
    arg:
        byer: byer i ruten
        size: størrelse på arraylist ut
    returns:
        vei_unix: liste med n(size) ruter, der ingen er duplikater
    '''
    
    lengde=len(byer)
    if lengde<8:
        vei_perm=np.array(list(permutations(byer)),dtype=np.int)
        np.random.shuffle(vei_perm)
        return vei_perm[0:size]
                            
    vei=np.array([np.random.permutation(byer) for _ in range(size)],dtype=np.int)
    vei_unik=np.unique(vei,axis=0)
    unik_len=vei_unik.shape[0]
    count=0
    while unik_len!=size:
        vei_ny=np.array([np.random.permutation(byer) for _ in range(size-unik_len)],dtype=np.int)
        vei=np.vstack([vei_unik,vei_ny])
        vei_unik=np.unique(vei,axis=0)
        unik_len=vei_unik.shape[0]
        count+=1
        
    return vei_unik

    




    



In [59]:
def hybrid(data,byer_list,marg=30,max_range=200):
    f_score=np.zeros(byer_list.shape[0],dtype=np.int)
    ny_perm=np.zeros(byer_list.size,dtype=np.int).reshape(byer_list.shape)
    orginal_perm=np.zeros(byer_list.size,dtype=np.int).reshape(byer_list.shape)

    for i,vei in enumerate(byer_list):
        score,ny_vei=hill_climb(data,byer=vei,margin=marg,max_range=max_range)
        f_score[i]=score
        ny_perm[i,:]=ny_vei
        orginal_perm[i,:]=vei
        
    fit_sort=np.argsort(f_score)
    gammel_veier_sortert=orginal_perm[fit_sort]
    ny_veier_sort=ny_perm[fit_sort]
    f_score_sort=f_score[fit_sort]

    return ny_veier_sort,gammel_veier_sortert,f_score_sort[0]

In [89]:
def genal(data,vei_lengde=6,byer=[],pop_size=40,cross_prosent=0.2,it=200,margin=50,md=3):
    sub_data=data.to_numpy()
    
    croos_antall=int(cross_prosent*pop_size)

    if croos_antall%2!=0:
        croos_antall+=1
    
    if len(byer)==0:
        byer=np.random.choice(24,vei_lengde,replace=False)
    else:
        vei_lengde=len(byer)
    
    
    #1 intiliaze population
    populasjon=popin(byer,pop_size)
    count=0
    beste_vei=[]
    beste_avstand=[]
    avarage_fitness=[]
    
    #loop
    #parents
    for c in range(it):
        gammel_best=populasjon[0,:]
        
        
        #crossover for første iterasjon bilr random, men så vil vi få crossover være blant de 20% beste

        #crossover
        kids=np.zeros(vei_lengde*int(croos_antall/2),dtype=np.int).reshape(int(croos_antall/2),vei_lengde)


        for i in range(0,croos_antall,2):
            kids[int(i/2),:]=croosover(populasjon[i,:].copy(),populasjon[i+1,:].copy())

        populasjon=np.r_[populasjon,kids]
        
        #mutasjon
        endret_pop=muta_list(populasjon)
        if endret_pop.shape[0]!=0:
            populasjon=np.r_[populasjon,endret_pop]
        
        #selection
        if md==0 or md==1:
            # lamar=0, baldwin=1
            temp_p=hybrid(data,populasjon)
            populasjon=temp_p[md]
            ny_best=populasjon[0,:]
            beste_vei.append(ny_best)
            beste_avstand.append(temp_p[2])
        
        else:
            populasjon=fitness(data,populasjon)
            #apped to list
            ny_best=populasjon[0,:]
            beste_avstand.append(avstand_fra_til(sub_data,ny_best))
            beste_vei.append(ny_best)
        
            
        populasjon_top=populasjon[0:croos_antall,:]
        populasjon_bot=populasjon[croos_antall:,:]
        np.random.shuffle(populasjon_bot)
        populasjon=np.r_[populasjon_top,populasjon_bot[0:pop_size-croos_antall,:]]
        
        

        
            
        if beste_avstand[-1]==avstand_fra_til(sub_data,gammel_best):
            count+=1
        else:
            count=0
            
        if count>margin: break
        
        print(f'{c}/{it}',end='\r')
       
    return beste_vei, beste_avstand
    



    
    
    
    
    
    

In [80]:
byer_ti=list(range(10))
ga_ti=genal(data,byer=byer_ti)[1]

print(f'beste avstand for exhaustive search for 10 byer er 7486, GA gir: {ga_ti[-1]} ')



beste avstand for exhaustive search for 10 byer er 7486, GA gir: 7486 


In [88]:
# For all three variants: As with the hill climber, report best, worst, mean
# and standard deviation of tour length out of 20 runs of the algorithm (of the best individual of last generation).
# Also, find and plot the average fitness of the best fit individual in each generation (average across runs),
# and include a figure with all three curves in the same plot in the report. Conclude which is best in terms of tour
# length and number of generations of evolution time.

avstander_10=[]
avstander_10_list=[]

avstander_24=[]
avstander_24_list=[]

rute_10=np.array([0,1,2,3,4,5,6,7,8,9])
rute_24=np.array(list(range(24)))

for i in range(20):
    ga_10=genal(data,byer=rute_10)[1]
    ga_24=genal(data,byer=rute_10)[1]
    avstander_10_list.append(ga_10)
    avstander_24_list.append(ga_24)
    avstander_10.append(ga_10[-1])
    avstander_24.append(ga_24[-1])

gjennomsnitt_10=statistics.mean(avstander_ti)
standardavvik_10=statistics.stdev(avstander_ti)
gjennomsnitt_24=statistics.mean(avstander_tjuefire)
standardavvik_24=statistics.stdev(avstander_tjuefire)


print('Ti ruter \n-------------------')
print(f'Gjennomsnitt: {int(gjennomsnitt_10)} \nStandardavik: {int(standardavvik_10)}\
    \nMinst: {min(avstander_ti)}\nMax: {max(avstander_ti)}')
print('\nTjuefire ruter \n-------------------')
print(f'Gjennomsnitt: {int(gjennomsnitt_24)} \nStandardavik: {int(standardavvik_24)}\
    \nMinst: {min(avstander_tjuefire)}\nMax: {max(avstander_tjuefire)}')






KeyboardInterrupt: 

Among the first 10 cities, did your GA find the shortest tour (as found by the exhaustive search)? Did it come close? 

For both 10 and 24 cities: How did the running time of your GA compare to that of the exhaustive search? 

How many tours were inspected by your GA as compared to by the exhaustive search?

## Hybrid Algorithm (IN4050 only)
### Lamarckian
Lamarck, 1809: Traits acquired in parents’ lifetimes can be inherited by offspring. In general the algorithms are referred to as Lamarckian if the result of the local search stage replaces the individual in the population.
### Baldwinian

Baldwin effect suggests a mechanism whereby evolutionary progress can be guided towards favourable adaptation without the changes in individual's fitness arising from learning or development being reflected in changed genetic characteristics. In general the algorithms are referred to as Baldwinian if the original member is kept, but has as its fitness the value belonging to the outcome of the local search process.


(See chapter 10 and 10.2.1 from Eiben and Smith textbook for more details. It will also be lectured in Lecure 4)

### Task
Implement a hybrid algorithm to solve the TSP: Couple your GA and hill climber by running the hill climber a number of iterations on each individual in the population as part of the evaluation. Test both Lamarckian and Baldwinian learning models and report the results of both variants in the same way as with the pure GA (min,
max, mean and standard deviation of the end result and an averaged generational plot). How do the results compare to that of the pure GA, considering the number of evaluations done?